In [1]:
import pandas as pd

df = pd.read_pickle('../data/final_data.pkl')

In [2]:
# MODEL PREDICTION 'Code_local_type' == 1, MAISON 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# MSE : 3 184 771 589 078.1714 avec 5 colonnes et 100 000 lignes au début du programme (régression linéaire)
# MSE : 9 351 187 946 604.617 avec 11 colonnes et 700 000 lignes au début du programme
# MSE : 9 621 056 123 391.139 en changeant le code type local == 1
# MSE : 2 642 213 419 979.8486 avec 5 colonnes et 100 000 lignes au début du programme (avec year, code postal, et nature mutation) (code local == 1)
# MSE : 2 642 223 129 457.908 avec pareil mais Lasso (few features should be important) (code local == 1)
# MSE : 2 283 126 024 532.157 avec pareil mais Ridge (lot of features should be important) (code local == 1)
# MSE : 1 481 548 894 581.531 avec Ridge mais 300 000 lignes (code local == 1)
# MSE : 1 419 765 756 499.442 avec Lasso mais 300 000 lignes (code local == 1)
# MSE : 15 011 506 135 748.002 avec Lasso mais 1 021 105 lignes (code local == 1)
# MSE : 6 011 506 135 748.002 avec Ridge mais ... lignes (code local == 1)

df_test = df.copy()
# test seulement avec le code local == 1
df_test = df_test[df_test['Code_type_local'] == 1]
df_test = df_test.drop(columns=["Code_type_local"])


print("nombre individus dataframe global : ", len(df_test), "\n")


# test drop colonne
X = df_test.drop(columns=['Valeur_fonciere', 'Commune', 'Section', 'day',
                 'No_plan', 'No_disposition', 'latitude', 'longitude', 'Code_postal', 'month'])
print(X.columns)

X = pd.get_dummies(X, columns=['year', 'Departement'])


y = df_test['Valeur_fonciere']  # Variable cible

# Séparez les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement et 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42)

# Initialisez le modèle Ridge
modele_ridge = Ridge(alpha=1.0)  # L'alpha est le paramètre de régularisation

modele_ridge.fit(X_train, y_train)
# Entraînez le modèle sur les donné
# Prédisez les valeurs sur l'ensemble de test
predictions = modele_ridge.predict(X_test)

# Calculez l'erreur quadratique moyenne (MSE) pour évaluer la performance du modèle
mse = mean_squared_error(y_test, predictions)

# Affichez l'erreur quadratique moyenne
print("Erreur Quadratique Moyenne (MSE) : ", mse)

# Créez un DataFrame avec les valeurs prédites et les valeurs réelles
resultats = pd.DataFrame(
    {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# Affichez les valeurs prédites par rapport aux valeurs réelles
print(resultats)
# Calculez le R carré
r2 = r2_score(y_test, predictions)
# Affichez le R carré
print("R carré :", r2)


nombre individus dataframe global :  31450 

Index(['Nombre_de_lots', 'Surface_reelle_bati', 'Nombre_pieces_principales',
       'year', 'Departement', 'Prix_moyen_m2'],
      dtype='object')
Erreur Quadratique Moyenne (MSE) :  5864823124.436262
       Valeur_reelle  Valeur_predite
1920        205000.0    95389.057829
47022         2200.0    87608.701572
26293        40000.0   104015.917400
38708       130000.0   105566.485030
19144       160000.0    73693.895077
...              ...             ...
35749       133300.0    54884.792886
37346         3000.0   102751.091496
40839        65100.0   139207.731742
32982        62000.0   135330.695239
3174         45000.0    35593.179448

[15725 rows x 2 columns]
R carré : 0.37934184683374095


In [3]:
# MODEL PREDICTION 'Code_local_type' == 2, APPARTEMENT

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score


df_test = df.copy()

# test seulement avec le code local == 2
df_test = df_test[df_test['Code_type_local'] == 2]
print(df['Code_type_local'].value_counts())

df_test = df_test.drop(columns=["Code_type_local"])

print("nombre individus dataframe global : ", len(df_test), "\n")


# test drop colonne
X = df_test.drop(columns=['Valeur_fonciere', 'Commune', 'Section', 'day',
                 'No_plan', 'No_disposition', 'latitude', 'longitude', 'Code_postal', 'month'])
print(X.columns)

X = pd.get_dummies(X, columns=['year', 'Departement'])

y = df_test['Valeur_fonciere']  # Variable cible

# Séparez les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement et 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Initialisez le modèle Ridge
modele_ridge = Ridge(alpha=1.0)  # L'alpha est le paramètre de régularisation

modele_ridge.fit(X_train, y_train)

# Entraînez le modèle sur les donné
# Prédisez les valeurs sur l'ensemble de test
predictions = modele_ridge.predict(X_test)

# Calculez l'erreur quadratique moyenne (MSE) pour évaluer la performance du modèle
mse = mean_squared_error(y_test, predictions)

# Affichez l'erreur quadratique moyenne
print("Erreur Quadratique Moyenne (MSE) : ", mse)

# Créez un DataFrame avec les valeurs prédites et les valeurs réelles
resultats = pd.DataFrame(
    {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# Affichez les valeurs prédites par rapport aux valeurs réelles
print(resultats)
# Calculez le R carré
r2 = r2_score(y_test, predictions)
# Affichez le R carré
print("R carré :", r2)


1.0    31450
2.0    10345
3.0     3363
4.0     1478
Name: Code_type_local, dtype: int64
nombre individus dataframe global :  10345 

Index(['Nombre_de_lots', 'Surface_reelle_bati', 'Nombre_pieces_principales',
       'year', 'Departement', 'Prix_moyen_m2'],
      dtype='object')
Erreur Quadratique Moyenne (MSE) :  4501936796.347327
       Valeur_reelle  Valeur_predite
4769         2640.00    73005.622833
45035       18994.64    59282.732451
17040        4060.00   105176.548579
13803       16000.00    57358.136169
11713       32000.00    44171.947748
...              ...             ...
42926      139500.00    46243.863941
2931        30000.00    47557.486250
4574        20000.00    15979.943103
31778      140000.00    92467.662940
26376      100000.00   151596.508438

[2069 rows x 2 columns]
R carré : 0.28335708388318337


In [4]:
# MODEL PREDICTION 'Code_local_type' == 3, DEPENDANCE


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score


df_test = df.copy()

# test seulement avec le code local == 3
df_test = df_test[df_test['Code_type_local'] == 3]
print(df['Code_type_local'].value_counts())

df_test = df_test.drop(columns=["Code_type_local"])

print("nombre individus dataframe global : ", len(df_test), "\n")


# test drop colonne
X = df_test.drop(columns=['Valeur_fonciere', 'Commune', 'Section', 'day',
                 'No_plan', 'No_disposition', 'latitude', 'longitude', 'Code_postal', 'month'])
print(X.columns)

X = pd.get_dummies(X, columns=['year', 'Departement'])

y = df_test['Valeur_fonciere']  # Variable cible

# Séparez les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement et 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Initialisez le modèle Ridge
modele_ridge = Ridge(alpha=1.0)  # L'alpha est le paramètre de régularisation

modele_ridge.fit(X_train, y_train)

# Entraînez le modèle sur les donné
# Prédisez les valeurs sur l'ensemble de test
predictions = modele_ridge.predict(X_test)

# Calculez l'erreur quadratique moyenne (MSE) pour évaluer la performance du modèle
mse = mean_squared_error(y_test, predictions)

# Affichez l'erreur quadratique moyenne
print("Erreur Quadratique Moyenne (MSE) : ", mse)

# Créez un DataFrame avec les valeurs prédites et les valeurs réelles
resultats = pd.DataFrame(
    {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# Affichez les valeurs prédites par rapport aux valeurs réelles
print(resultats)
# Calculez le R carré
r2 = r2_score(y_test, predictions)
# Affichez le R carré
print("R carré :", r2)


1.0    31450
2.0    10345
3.0     3363
4.0     1478
Name: Code_type_local, dtype: int64
nombre individus dataframe global :  3363 

Index(['Nombre_de_lots', 'Surface_reelle_bati', 'Nombre_pieces_principales',
       'year', 'Departement', 'Prix_moyen_m2'],
      dtype='object')
Erreur Quadratique Moyenne (MSE) :  6749516335.401878
       Valeur_reelle  Valeur_predite
13799       118000.0   107001.481791
30246        75000.0   107448.876552
9866         14000.0    92033.402161
37907       129000.0    77797.747791
2513        280000.0   111961.509553
...              ...             ...
24034       331200.0   151523.864002
36491        15000.0    80555.183938
15025        95000.0    55771.656465
41498         6000.0    86168.682623
36344         4500.0   222171.530385

[673 rows x 2 columns]
R carré : 0.23799857797032742
